<a href="https://colab.research.google.com/github/358Xin/DL/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip
!unzip -q food-11.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
To: /content/food-11.zip
100% 963M/963M [00:05<00:00, 170MB/s]


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, Dataset, Subset
from torchvision.datasets import DatasetFolder
from tqdm.auto import tqdm

In [3]:
train_transform = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()])

In [4]:
from torch.functional import broadcast_shapes
from torch.utils.data.dataloader import DataLoader
batch_size = 128

train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_transform)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_transform)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_transform)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.opne(x), extensions="jpg", transform=test_transform)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [5]:
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.conv import Conv2d
from torch.nn.modules.batchnorm import BatchNorm2d
class Classifier:
  def __init__(self):
    super(Classifier, self).__init__()

    self.cnn_layers = nn.Sequential(
        nn.Conv2d(3, 64, 3, 1, 1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2,2,0),

        nn.Conv2d(64, 128, 3, 1, 1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2, 2, 0),

        nn.Conv2d(128, 256, 3, 1, 1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(4, 4, 0)
    )

    self.fc_layers = nn.Sequential(
        nn.Linear(256*8*8, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 11)
    )
  
  def forward(self, x):
    x = self.cnn_layers(x)
    x = x.flatten(1)
    x = self.fc_layers(x)
    return x

In [6]:
def get_pseudo_labels(dataset, model, threshold=0.65):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  
  data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

  model.eval()
  softmax = nn.Softmax(-1)
  for batch in tqdm(data_loader):
    img, _ = batch
    with torch.no_grad():
      logits = model(img.to(device))
    probs = softmax(logits)
  model.train()
  return dataset

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Classifier().to(device)
model.device = device

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
n_epochs = 80
do_semi = False

for epoch in range(n_epochs):
  if do_semi:
    pseudo_set = get_pseudo_labels(unlabeled_set, model)
    concat_dataset = ConcatDataset([train_set, pseudo_set])
    train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
  model.train()
  train_loss = []
  train_acc = []

  for batch in tqdm(train_loader):
    img, label = batch
    logits = model(img.to(device))
    loss = criterion(logits, label.to(device))
    optimizer.zero_grad()
    loss.backward()
    